# Imports, Definitions, and Instantiating the overlay

In [1]:
import logging
import numpy as np
import time
import itertools
import matplotlib.pyplot as plt
import scipy.signal
from logging import getLogger
import os
from glob import glob

import bitstruct
from fpbinary import FpBinary, OverflowEnum, RoundingEnum
from mkidgen3.daccomb import generate as gen_comb
from mkidgen3.testutils import *
import mkidgen3.testutils as tu
from mkidgen3.fixedpoint import *
from mkidgen3.pynq import dma_status  ## also activates the drivers
import mkidgen3.pynq as mp

import xrfdc, xrfclk
import pynq
from pynq import PL, Overlay, DefaultIP, allocate, DefaultHierarchy


matched_filter_loaded=False
logging.basicConfig()
logging.getLogger('').setLevel('DEBUG')
logging.getLogger('__main__').setLevel('INFO')

Load the overlay and extract the dma core

In [2]:
ol = Overlay('/home/xilinx/gen3/test_iqadccap/design_1.bit', ignore_version=True)
ol.download()
print(f"PL Bitfile: {PL.bitfile_name}\nPL Timestamp: {PL.timestamp}\n"
      f"Overlay timestamp: {ol.timestamp}  Loaded: {ol.is_loaded()}")

DEBUG:asyncio:Using selector: EpollSelector


PL Bitfile: /home/xilinx/gen3/test_iqadccap/design_1.bit
PL Timestamp: 2021/2/26 16:39:7 +981948
Overlay timestamp: 2021/2/26 16:39:7 +981948  Loaded: True


In [3]:
iqgen=ol.iq_gen_0
iqc0=ol.iq_capture_0
rfdc=ol.usp_rf_data_converter_0
ddr=ol.ddr4_0
adc0=ol.adc_capture_0

In [4]:
for ip, d in ol.ip_dict.items():
    print(ip, d['type'])

adc_capture_0 mazinlab:mkidgen3:adc_capture:0.7
iq_capture_0 mazinlab:mkidgen3:iq_capture:0.8
iq_gen_0 mazinlab:mkidgen3:iq_gen:0.1
usp_rf_data_converter_0 xilinx.com:ip:usp_rf_data_converter:2.3
ddr4_0 xilinx.com:ip:ddr4:2.2
zynq_ultra_ps_e_0 xilinx.com:ip:zynq_ultra_ps_e:3.3


# Set up the PL DDR4 MMIO

In [5]:
ddr4_mmio=pynq.MMIO(mp.PL_DDR4_ADDR, length=2**32)

In [6]:
ddr4_mmio.array[:]=1

# Configure IQ cap

Each group is 32 bytes (8 complex shorts). DDR4 saturation should occur if n_grps>77 while adc cap is running.

In [7]:
n_cap = 400000
grps = list(range(16))+list(range(32,48))
grps = list(range(32))

iq_mb = n_cap*len(grps)*32/1024**2
iq_mbps = 32*512*len(grps)/256
iq_eta = n_cap/2/1024**2*1000
iqc0.capture(n_cap, groups=grps, start=False)

Will capture ~390.625 MB of data @ 2048.0 MBps. ETA 191 ms


In [8]:
iqc0.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  keep_1 = Register(keep=4294967295),
  keep_2 = Register(keep=0),
  keep_3 = Register(keep=0),
  keep_4 = Register(keep=0),
  keep_5 = Register(keep=0),
  keep_6 = Register(keep=0),
  keep_7 = Register(keep=0),
  keep_8 = Register(keep=0),
  total_capturesize_1 = Register(total_capturesize=102400000),
  total_capturesize_2 = Register(total_capturesize=0),
  capturesize = Register(capturesize=12800000),
  iqout_1 = Register(iqout=0),
  iqout_2 = Register(iqout=4)
}

# Configure IQ Generator

In [9]:
iqgen.generate((n_cap+2)*256)
iqgen.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  max = Register(max=102400512)
}

# Configure ADC Cap

In [18]:
n_cap = 40000000*3
adc_mb=n_cap*32/1024**2
adc_mbps=32*512
adc_eta=n_cap/512e3
print(f"Will capture {adc_mb:.0f} MB of data @ {adc_mbps} MBps. ETA {adc_eta:.0f} ms")
adc0.capture(n_cap, start=False, addr=0*2**31/2**12)

Will capture 3662 MB of data @ 16384 MBps. ETA 234 ms


In [11]:
adc0.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  capturesize = Register(capturesize=40000000),
  iqout_1 = Register(iqout=2147483648),
  iqout_2 = Register(iqout=4)
}

In the test design the replay outputs 16 IQ values each tick and IQ Capture 0 gets the lower 8. So if we replay 0,1,2.... the groups of 8 it gets will be 0-7, 16-23, etc. So groups 0-16 

# Run It 

In [12]:
ddr_mbps = 333*512/8
print(f'Will use {(adc_mbps+iq_mbps)/ddr_mbps*100:.2f}% of DDR4 bandwidth.')

Will use 86.49% of DDR4 bandwidth.


In [13]:
iqgen.register_map.CTRL.AP_START=1

In [14]:
tic=time.time()
iqc0.register_map.CTRL.AP_START=1
adc0.register_map.CTRL.AP_START=1
toc=time.time()
print(f'Started in {(toc-tic)*1000:.1} ms')

Started in 0.6 ms


In [15]:
iqgen.register_map, iqc0.register_map, adc0.register_map

(RegisterMap {
   CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
   GIER = Register(Enable=0, RESERVED=0),
   IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
   IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
   max = Register(max=102400512)
 }, RegisterMap {
   CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
   GIER = Register(Enable=0, RESERVED=0),
   IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
   IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
   keep_1 = Register(keep=4294967295),
   keep_2 = Register(keep=0),
   keep_3 = Register(keep=0),
   keep_4 = Register(keep=0),
   keep_5 = Register(keep=0),
   keep_6 = Register(keep=0),
   keep_7 = Register(keep=0),
   keep_8 = Register(keep=0),
   total_capturesize_1 = Register(total_capturesize=102400000),
   total_capturesize_2 = Register(total_capturesize=0

In [20]:
adc0.register_map.CTRL.AP_START=1

In [ ]:
ddr4_mmio.array[ddr4_mmio.array.size//2]

In [ ]:
x=ddr4_mmio.array[:8*(n_cap+2)*len(grps)]
x=x.reshape((x.size//8,8)).copy()

In [ ]:
hex(ddr4_mmio.array.size//2*4)

In [ ]:
x[:,0].tolist()

In [ ]:
ddr4_mmio.array[10000000]